In [54]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import timeit
from models import *
from equations import *
from solvers import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
data_file_name = "data.npz"
params = ModelParams.load_from_npz(data_file_name)
model = Model(params)
shocks = model.shocks

In [56]:
model.params.tilde_tau.max()

np.float64(1.915345393)

In [57]:
solver = ModelSolver(model)
solver.solve()

Iter 1: w_min=9.947e-01, w_max=1.002e+00, X_min=1.760e+03, X_max=4.828e+09, Δw=5.310e-03, ΔP=2.220e-16
Iter 2: w_min=9.906e-01, w_max=1.003e+00, X_min=1.759e+03, X_max=4.833e+09, Δw=4.097e-03, ΔP=3.585e-03
Iter 3: w_min=9.878e-01, w_max=1.004e+00, X_min=1.758e+03, X_max=4.838e+09, Δw=2.755e-03, ΔP=3.442e-03
Iter 4: w_min=9.862e-01, w_max=1.004e+00, X_min=1.757e+03, X_max=4.842e+09, Δw=1.645e-03, ΔP=2.808e-03
Iter 5: w_min=9.853e-01, w_max=1.004e+00, X_min=1.756e+03, X_max=4.845e+09, Δw=8.649e-04, ΔP=1.966e-03
Iter 6: w_min=9.849e-01, w_max=1.004e+00, X_min=1.756e+03, X_max=4.847e+09, Δw=4.401e-04, ΔP=1.227e-03
Iter 7: w_min=9.848e-01, w_max=1.005e+00, X_min=1.755e+03, X_max=4.849e+09, Δw=2.945e-04, ΔP=7.866e-04
Iter 8: w_min=9.848e-01, w_max=1.005e+00, X_min=1.755e+03, X_max=4.849e+09, Δw=2.733e-04, ΔP=4.984e-04
Iter 9: w_min=9.849e-01, w_max=1.005e+00, X_min=1.755e+03, X_max=4.850e+09, Δw=1.996e-04, ΔP=3.103e-04
Iter 10: w_min=9.850e-01, w_max=1.005e+00, X_min=1.755e+03, X_max=4.850e+

In [58]:
model.sol.w_hat

array([1.00487287, 1.00025114, 0.99697138, 0.99321673, 0.99808464,
       0.99595041, 0.99338436, 0.98885243, 0.99426271, 0.99663022,
       0.99658891, 0.99684319, 0.99505486, 0.99818777, 0.99951223,
       0.99243071, 0.99628666, 1.00057927, 0.9966398 , 0.99593293,
       0.99722894, 1.00065737, 0.99301181, 1.00293684, 0.99960175,
       0.99439621, 0.98737658, 0.98508414, 1.0022247 , 0.99773342,
       0.99672645, 0.99637068, 0.99658095, 0.99559804, 0.99802976,
       0.99610933, 1.00355008])

In [9]:
model.sol.Xf_prime.shape

(37, 23)

In [11]:
model.params.pif.shape

(37, 37, 23)